In [14]:
import json
from pprint import pprint
import csv
import urllib
import os
import re
import sys 
import requests
reload(sys)
sys.setdefaultencoding('UTF-8')

commSite = 'http://www.dailymail.co.uk/reader-comments/p/asset/readcomments/'
start = '?max=100&offset='
end ='&order=desc&rcCache=shout'
headers = 'id', 'dateCreated', 'message', 'assetId', 'assetUrl','assetCommentCount', 'userAlias','userLocation','assetHeadline',   'userIdentifier', 'voteRating', 'voteCount', 'hasProfilePicture'
headersReq = {
    'Host': 'www.dailymail.co.uk',
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Encoding': 'gzip, deflate, sdch',
    'Accept-Language': 'en-US,en;q=0.8',
}

URL = raw_input('Enter Full Daily Mail Article URL: ')
#URL = 'http://www.dailymail.co.uk/news/article-4187460/Lovre-evacuated-Paris.html'
article = re.search('\/article-(\d+)\/',URL).group(1)

outfile  = open(article+'.csv', "wb")
writer = csv.writer(outfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
writer.writerow(headers)

reqURL = commSite + article
req = requests.get (reqURL, headers = headersReq)
html_source = req.text

numOfComments = re.findall ('{"total":(\d+)',html_source)
nums = int(numOfComments[0])

#Get comments
for i in range (0, nums,100): 
    comment = []
    req = requests.get (reqURL + start + str(i) + end, headers = headersReq)
    data = json.loads(req.text)
    for i in range(len(data['payload']['page'])):
        for header in headers:
            comment.append(data['payload']['page'][i].get(header,'N\\A'))
            print '\n'*3
        writer.writerow(comment)
        print data['payload']['page'][i]['message'].encode("UTF-8")
        print data['payload']['page'][i]['voteRating']
        comment = []
        
outfile.close()
      
req.connection.close()

print 'Done!'

Enter Full Daily Mail Article URL: http://www.dailymail.co.uk/news/article-4187460/Lovre-evacuated-Paris.html
